In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import set_config
set_config(display = 'diagram')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
filename = '/content/drive/My Drive/Cereal with missing values.xlsx'
df = pd.read_excel(filename, index_col = 'name')
df.head()
#¿Qué tan bien se pueden predecir las calorías basándose en el fabricante, tipo de cereal, gramos de grasa, gramos de azúcares y el peso en onzas por una porción del cereal? 

,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
name,,,,,,,,,,,,,,,
Apple Cinnamon Cheerios,General Mills,Cold,110.0,2.0,2.0,180.0,1.5,10.5,10.0,70.0,25.0,1.0,1.00,0.75,29.509541
Basic 4,General Mills,Cold,130.0,3.0,2.0,NaN,2.0,18.0,NaN,100.0,25.0,3.0,1.33,0.75,37.038562
Cheerios,General Mills,Cold,NaN,6.0,2.0,290.0,2.0,17.0,1.0,105.0,25.0,1.0,1.00,1.25,50.764999
Cinnamon Toast Crunch,General Mills,Cold,120.0,1.0,3.0,210.0,0.0,13.0,9.0,45.0,25.0,2.0,1.00,0.75,19.823573
Clusters,General Mills,Cold,110.0,3.0,2.0,140.0,2.0,13.0,7.0,105.0,25.0,3.0,1.00,0.50,40.400208


In [15]:
# Primero buscamos duplicados

print('Hay', df.duplicated().sum(), 'duplicados.')


Hay 0 duplicados.


In [16]:
# Segundo buscamos datos que faltan

print('Hay', df.isna().sum().sum(), 'datos que faltan')

Hay 35 datos que faltan


In [17]:
# Listar por columna
df.isna().sum()

Manufacturer                                       0
type                                               9
calories per serving                               7
grams of protein                                   0
grams of fat                                       8
milligrams of sodium                               1
grams of dietary fiber                             0
grams of complex carbohydrates                     0
grams of sugars                                    9
milligrams of potassium                            0
vitamins and minerals (% of FDA recommendation)    1
Display shelf                                      0
Weight in ounces per one serving                   0
Number of cups in one serving                      0
Rating of cereal                                   0
dtype: int64

In [35]:
# Damos de baja los nan que hay en nuestro objetivo.
df.dropna(subset = ['calories per serving'], inplace = True)

In [19]:
# Verificar que si se eliminaron
print('Hay', df['calories per serving'].isna().sum(), 'datos de calorias que faltan')

Hay 0 datos de calorias que faltan


In [20]:
#Identificar cada característica como numérica, ordinal o nominal. (Entreguen esta pregunta en una celda de texto en un cuaderno de Colab).

print(df.select_dtypes(include = 'object').info())
print(df.select_dtypes(include = 'number').info())

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, Apple Cinnamon Cheerios to Quaker Oatmeal
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Manufacturer  70 non-null     object
 1   type          62 non-null     object
dtypes: object(2)
memory usage: 1.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, Apple Cinnamon Cheerios to Quaker Oatmeal
Data columns (total 13 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   calories per serving                             70 non-null     float64
 1   grams of protein                                 70 non-null     float64
 2   grams of fat                                     62 non-null     float64
 3   milligrams of sodium                             69 non-null     float64
 4   grams of dietary fiber                           70

La manufacturer y la type son nominales y el resto son numéricas: 
-calories per serving
-grams of fat
-milligrams of sodium
-grams of dietary fiber
-grams of sugars
-milligrams of potassium
-vitamins and minerals
-Weight in ounces per one serving
-Number of cups in one serving
-Rating of cereal

In [21]:
#Definir la característica (X) y objetivo (y).

característica = ['Manufacturer', 
            'type', 
            'grams of fat', 
            'grams of sugars',
            'Weight in ounces per one serving']
X = df[característica]
y = df['calories per serving']

In [22]:
#Utilizar train test split en los datos para prepararlos para aprendizaje automático.

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [23]:
# Utilizar transformadores de columnas:select
num_selector = make_column_selector(dtype_include = 'number')
cat_selector = make_column_selector(dtype_include = 'object')

# Utilizar transformadores de columnas:trans. 
scaler = StandardScaler()
ohe = OneHotEncoder(sparse = False, handle_unknown = 'ignore')


In [24]:
# Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas y la estrategia “most_frequent” para las columnas categóricas.
mean_imp = SimpleImputer(strategy = 'mean')
freq_imp = SimpleImputer(strategy = 'most_frequent')

# Utilizar pipelines
num_pipeline = make_pipeline(mean_imp, scaler)
cat_pipeline = make_pipeline(freq_imp, ohe)

In [29]:
# Hacer tuplas
num_tuple = (num_pipeline, num_selector)
cat_tuple = (cat_pipeline, cat_selector)

# APLICAR
preprocesamiento = make_column_transformer(num_tuple, cat_tuple)

In [31]:
# Todos los datos de preprocesamiento deben estar contenidos en un único objeto de preprocesamiento.
preprocesamiento.fit(X_train)

# APLICAR
X_train_processed = preprocesamiento.transform(X_train)
X_test_processed = preprocesamiento.transform(X_test)

In [32]:
# evitar la fuga de datos para dejarlos listos para el modelado.
print('Hay',  np.isnan(X_train_processed).sum().sum(), 'datos que faltan en el training.')
print('Hay', np.isnan(X_test_processed).sum().sum(), 'datos que faltan en el set.')


Hay 0 datos que faltan en el training.
Hay 0 datos que faltan en el set.


In [40]:
# Exp de los datos

print(X_train_processed)
print ('------')
print(X_test_processed)

[[-1.00539366e+00 -1.53835815e+00 -3.52083059e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-4.10364759e-02  1.19210391e+00 -2.39769825e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-4.10364759e-02 -8.55742637e-01 -2.39769825e-01  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 9.23320709e-01  5.44113816e-02  1.92573028e+00  0.00000000e+00
   1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00539366e+00 -1.53835815e+00 -1.35533049e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00539366e+00  5.44113816e-02 -2.39769825e-01  0.00000000e+00
   1.00000000e+00  